## Context:

This was a very simple code that I made to fill a data conversion need. 

Data came in from a tool in this format: POLYGON ((-80.6166339 25.2756544, -65.8070636 18.4287094, -64.7523761 32.0929608, -80.6166339 25.2756544))

And we needed to work with it in this JSON format: {"geometry":{"type":"Polygon","coordinates":[[[-80.6166339,25.2756544],[-65.8070636,18.4287094],[-64.7523761,32.0929608],[-80.6166339,25.2756544]]]}}

It does some hacky string manipulation, but it works and it is used to process hundreds of records like this every month.

The data here is kept very high level to show the general way it is done, but for company privacy does not give too much detail.

In [20]:
import pandas as pd

In [24]:
file_path = # add file path here

df = pd.read_csv(file_path)


In [26]:
df.head()

,WKT,name,description
0,"POLYGON ((-80.6166339 25.2756544, -65.8070636 ...",Bermuda Triangle,NaN


In [28]:
def raw_to_json_converter(raw_data):
    formatted_coordinates = raw_data[8:].replace(", ","],[")
    formatted_coordinates = formatted_coordinates.replace(" ",",")
    formatted_coordinates = formatted_coordinates.replace("((","[[[")
    formatted_coordinates = formatted_coordinates.replace("))","]]]")
    formatted_text = f'*"geometry":*"type":"Polygon","coordinates":{formatted_coordinates}%%'
    formatted_text = formatted_text.replace("*","{")
    formatted_text = formatted_text.replace("%","}")

    return formatted_text
    

In [30]:
# Apply the function to data and get the formatted data in a new column

df['formatted_shape'] = df.apply(lambda row: raw_to_json_converter(row['WKT']),axis=1)

In [32]:
df.head()

,WKT,name,description,formatted_shape
0,"POLYGON ((-80.6166339 25.2756544, -65.8070636 ...",Bermuda Triangle,NaN,"{""geometry"":{""type"":""Polygon"",""coordinates"":[[..."


In [34]:
output_file_name = 'output.csv'

df.to_csv(output_file_name,index=False)